<a href="https://colab.research.google.com/github/Jang-Sohyun/DACON/blob/main/%EA%B0%90%EA%B7%A4_%EC%B0%A9%EA%B3%BC%EB%9F%89_%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8(%EC%88%98%EC%A0%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Import

In [ ]:
!pip3 install pytorch_tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 1.1 MB/s eta 0:00:00


In [ ]:
import os
import torch
import random

import pandas as pd
import numpy as np

from sklearn.model_selection import KFold

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def nmae_score(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

def xgb_nmae(pred, dtrain):
    true = dtrain.get_label()
    return "nmae", nmae_score(true, pred)

# 1. 데이터 불러오기

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/4J DACON/감귤 착과량 예측 AI 경진대회/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/4J DACON/감귤 착과량 예측 AI 경진대회/test.csv")

print(df_train.info())
print(df_train.isnull().sum())

print(df_test.info())
print(df_test.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2207 entries, 0 to 2206
Columns: 184 entries, ID to 2022-11-28 엽록소
dtypes: float64(182), int64(1), object(1)
memory usage: 3.1+ MB
None
ID                0
착과량(int)          0
수고(m)             0
수관폭1(min)         0
수관폭2(max)         0
                 ..
2022-11-24 엽록소    0
2022-11-25 엽록소    0
2022-11-26 엽록소    0
2022-11-27 엽록소    0
2022-11-28 엽록소    0
Length: 184, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Columns: 183 entries, ID to 2022-11-28 엽록소
dtypes: float64(182), object(1)
memory usage: 3.1+ MB
None
ID                0
수고(m)             0
수관폭1(min)         0
수관폭2(max)         0
수관폭평균             0
                 ..
2022-11-24 엽록소    0
2022-11-25 엽록소    0
2022-11-26 엽록소    0
2022-11-27 엽록소    0
2022-11-28 엽록소    0
Length: 183, dtype: int64


# 2. 파생 변수 생성

In [ ]:
# 함수 정의 : df_v1 ~ df_v9까지 원하는 데이터 추출할 수 있음
def fn_get_feature_v3(df, idx=2) :    
    idx = 2 - idx
    if idx > 7 :
        idx = 6
    
    # 파생 변수
    df = df.copy()
    df['새순mean'] = df.iloc[:,6-idx:95-idx].mean(axis=1)
    df['새순std'] = df.iloc[:,6-idx:95-idx].std(axis=1)
    df['새순min'] = df.iloc[:,6-idx:95-idx].min(axis=1)
    df['새순max'] = df.iloc[:,6-idx:95-idx].max(axis=1)
    df['새순gap'] = df.iloc[:,94-idx] - df.iloc[:,6-idx]
    
    df['엽록소mean'] = df.iloc[:,95-idx:184-idx].mean(axis=1)
    df['엽록소std'] = df.iloc[:,95-idx:184-idx].std(axis=1)
    df['엽록소min'] = df.iloc[:,95-idx:184-idx].min(axis=1)
    df['엽록소max'] = df.iloc[:,95-idx:184-idx].max(axis=1)
    df['엽록소gap'] = df.iloc[:,183-idx] - df.iloc[:,95-idx]  
    

    df_v1 = df.iloc[:,1-idx:6-idx] # 시계열 데이터 제외한 학습 데이터
    df_v2 = df.iloc[:,1-idx:184-idx] # 원본 데이터
    df_v3 = pd.concat([df.iloc[:,1-idx:6-idx], df.iloc[:,184-idx:]], axis=1) # 파생변수 포함한 학습 데이터 (세순 및 엽록소 시계열 데이터 제외)
    df_v4 = df.iloc[:,1-idx:] # 파생변수까지 모두 포함한 학습 데이터
    df_v5 = pd.concat([df.iloc[:,1-idx:6-idx], df.iloc[:,184-idx:189-idx]], axis=1) # 시계열 데이터 제외, 새순 관련 파생병수만 포함된 학습 데이터
    df_v6 = df.iloc[:,1-idx:189-idx] # 원본데이터에 새순 관련 파생변수만 포함한 학습 데이터
    
    df_v7 = pd.concat([df.iloc[:,1-idx:95-idx], df.iloc[:,184-idx:189-idx]], axis=1) # 수고,수관폭 및 세순관련 시계열 데이터 + 파생변수만 포함된 학습 데이터
    df_v8 = pd.concat([df.iloc[:,1-idx:95-idx], df.iloc[:,184-idx:]], axis=1) # 수고,수관폭, 세순관련 시계열 데이터, 세순 관련 파생변수, 엽록소 관련 파생변수가 포함된 학습 데이터
    
    df_v9 = pd.concat([df.iloc[:,1-idx], df.iloc[:,6-idx:95-idx]], axis=1)
    df_v9 = pd.concat([df_v9, df.iloc[:,184-idx:189-idx]], axis=1) # 새순관련 시계열 데이터 및 세순 관련 파생변수만 포한된 학습데이터
    

    return df_v1, df_v2, df_v3, df_v4, df_v5, df_v6, df_v7, df_v8, df_v9

In [ ]:
# 첫번째 함수로 파생변수 생성
df_train_v1, df_train_v2, df_train_v3, df_train_v4, df_train_v5, df_train_v6, df_train_v7, df_train_v8, df_train_v9 = fn_get_feature_v3(df_train, idx=2)
df_test_v1, df_test_v2, df_test_v3, df_test_v4, df_test_v5, df_test_v6, df_test_v7, df_test_v8, df_test_v9 = fn_get_feature_v3(df_test, idx=1)
df_train_v2.head(2) # 원본 데이터

,착과량(int),수고(m),수관폭1(min),수관폭2(max),수관폭평균,2022-09-01 새순,2022-09-02 새순,2022-09-03 새순,2022-09-04 새순,2022-09-05 새순,...,2022-11-19 엽록소,2022-11-20 엽록소,2022-11-21 엽록소,2022-11-22 엽록소,2022-11-23 엽록소,2022-11-24 엽록소,2022-11-25 엽록소,2022-11-26 엽록소,2022-11-27 엽록소,2022-11-28 엽록소
0,692,275.0,287.0,292.0,289.5,2.8,2.8,2.7,2.7,2.7,...,70.978249,70.876794,70.705253,70.559603,70.427356,70.340491,70.29383,70.262422,70.169841,70.043251
1,534,293.0,284.0,336.0,310.0,3.3,3.3,3.3,3.2,3.2,...,71.535483,71.382303,71.253604,71.092665,70.955608,70.796630,70.59755,70.565088,70.560502,70.427600


In [ ]:
df_train_v3.head() # 파생변수 포함한 학습 데이터 (새순 및 엽록소 시계열 데이터 제외)

,착과량(int),수고(m),수관폭1(min),수관폭2(max),수관폭평균,새순mean,새순std,새순min,새순max,새순gap,엽록소mean,엽록소std,엽록소min,엽록소max,엽록소gap
0,692,275.0,287.0,292.0,289.5,1.693258,0.638143,0.6,2.8,-2.2,74.155836,2.427798,70.043251,78.336504,-8.293252
1,534,293.0,284.0,336.0,310.0,1.861798,0.894491,0.0,3.3,-3.3,74.962123,2.813831,70.427600,80.116691,-9.689090
2,634,300.0,392.0,450.0,421.0,1.762921,0.712611,0.5,3.0,-2.5,74.727999,2.476758,70.399578,79.118529,-8.718951
3,639,289.0,368.0,379.0,373.5,1.857303,0.692802,0.7,3.1,-2.4,73.546210,2.641605,69.133970,77.936262,-8.802293
4,496,306.0,353.0,358.0,355.5,2.125843,0.898171,0.6,3.7,-3.1,71.841067,2.642378,67.410093,76.233231,-8.823138


# 3. 모델링
- 새순 관련 데이터만 사용(df_train_V9)

In [ ]:
labels = df_train_v9["착과량(int)"]
feature_columns = df_train_v9.loc[:, "2022-09-01 새순":"2022-11-28 새순"].columns 

print("학습에는 새순 Features 89개를 사용했습니다.\n")
print(feature_columns)

학습에는 새순 Features 89개를 사용했습니다.

Index(['2022-09-01 새순', '2022-09-02 새순', '2022-09-03 새순', '2022-09-04 새순',
       '2022-09-05 새순', '2022-09-06 새순', '2022-09-07 새순', '2022-09-08 새순',
       '2022-09-09 새순', '2022-09-10 새순', '2022-09-11 새순', '2022-09-12 새순',
       '2022-09-13 새순', '2022-09-14 새순', '2022-09-15 새순', '2022-09-16 새순',
       '2022-09-17 새순', '2022-09-18 새순', '2022-09-19 새순', '2022-09-20 새순',
       '2022-09-21 새순', '2022-09-22 새순', '2022-09-23 새순', '2022-09-24 새순',
       '2022-09-25 새순', '2022-09-26 새순', '2022-09-27 새순', '2022-09-28 새순',
       '2022-09-29 새순', '2022-09-30 새순', '2022-10-01 새순', '2022-10-02 새순',
       '2022-10-03 새순', '2022-10-04 새순', '2022-10-05 새순', '2022-10-06 새순',
       '2022-10-07 새순', '2022-10-08 새순', '2022-10-09 새순', '2022-10-10 새순',
       '2022-10-11 새순', '2022-10-12 새순', '2022-10-13 새순', '2022-10-14 새순',
       '2022-10-15 새순', '2022-10-16 새순', '2022-10-17 새순', '2022-10-18 새순',
       '2022-10-19 새순', '2022-10-20 새순', '2022-10-21 새순', '2022-10-22

## 3-1. XGBoost

### 변수 중요도

In [ ]:
FOLDS = 5 
kfold = KFold(n_splits=FOLDS, random_state=42, shuffle=True)

feature_importance_df = pd.DataFrame()
oof = np.zeros(len(df_train_v9))
pred = np.zeros(len(df_test_v9))

for trn_idx, val_idx in kfold.split(df_train_v9):
    # k-fold 방식으로 train set과 valid set 분류
    X_train = df_train_v9.loc[trn_idx, feature_columns].reset_index(drop=True)
    y_train = df_train_v9.loc[trn_idx, "착과량(int)"].reset_index(drop=True)
    X_valid = df_train_v9.loc[val_idx, feature_columns].reset_index(drop=True)
    y_valid = df_train_v9.loc[val_idx, "착과량(int)"].reset_index(drop=True)
    
    # 위에서 분류된 data set을 반복적으로 xgbregressor 모델에 적용
    lr = XGBRegressor(
        objective="reg:squarederror",
        n_estimators=50000,
        learning_rate=0.01,
        subsample=0.7,
        colsample_bytree=0.3,
        random_state=42,
        )
    lr.fit(X_train, y_train, 
    eval_set=[(X_train, y_train), (X_valid, y_valid)], 
    eval_metric=xgb_nmae, 
    early_stopping_rounds=200,
    verbose=False)
    
    # valid data에 대한 예측값을 oof에 저장, test에 대한 예측값을 pred에 저장, feature_importance_df에 feature와 importance 저장
    oof[val_idx] = lr.predict(X_valid)
    pred += lr.predict(df_test_v9[feature_columns]) / FOLDS
    feature_importance_df = pd.concat([feature_importance_df, 
                                       pd.DataFrame(zip(feature_columns, lr.feature_importances_), columns=["feature", "importance"])])
else:
    feature_importance_df = feature_importance_df.groupby("feature").mean().sort_values("importance", ascending=False).reset_index()
    print("새순 Features 중 착과량에 가장 큰 영향을 주는 일자는 대부분 9월에 집중되어 있었음\n")
    print(feature_importance_df.head(10)) 
    print(nmae_score(labels, oof)), 
    print(mean_absolute_error(labels, oof))

새순 Features 중 착과량에 가장 큰 영향을 주는 일자는 대부분 9월에 집중되어 있었음

         feature  importance
0  2022-09-07 새순    0.134390
1  2022-09-16 새순    0.087047
2  2022-09-02 새순    0.081443
3  2022-09-05 새순    0.076677
4  2022-09-11 새순    0.073762
5  2022-09-14 새순    0.056333
6  2022-09-10 새순    0.050703
7  2022-09-15 새순    0.042787
8  2022-09-19 새순    0.039300
9  2022-09-04 새순    0.036135
0.07382409476729464
29.98900641323378


In [ ]:
best_score = 999999
best_cnt = 999999

print("사용한 새순 Features 개수별 validation score")

for cnt in [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]: # 사용할 Features 개수 임의 지정
    feature_columns = feature_importance_df.iloc[:cnt, 0].tolist() # 중요 변수 중 상위 몇 개까지 사용할지
    oof = np.zeros(len(df_train_v9))
    pred = np.zeros(len(df_test_v9))

    for trn_idx, val_idx in kfold.split(df_train_v9):
        X_train = df_train_v9.loc[trn_idx, feature_columns].reset_index(drop=True)
        y_train = df_train_v9.loc[trn_idx, "착과량(int)"].reset_index(drop=True)
        X_valid = df_train_v9.loc[val_idx, feature_columns].reset_index(drop=True)
        y_valid = df_train_v9.loc[val_idx, "착과량(int)"].reset_index(drop=True)

        lr = XGBRegressor(
            objective="reg:squarederror",
            n_estimators=50000,
            learning_rate=0.01,
            subsample=0.7,
            colsample_bytree=0.3,
            random_state=42,
            )
        lr.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_valid, y_valid)], 
        eval_metric=xgb_nmae, 
        early_stopping_rounds=200, 
        verbose=False)
        
        oof[val_idx] = lr.predict(X_valid)
        pred += lr.predict(df_test_v9[feature_columns]) / FOLDS

    
    print("Feature 수 :", cnt, "/ 결과 :", mean_absolute_error(labels, oof))

    if best_score>mean_absolute_error(labels, oof):
        best_score = mean_absolute_error(labels, oof)
        best_cnt = cnt
        oof_xgb = oof.copy()
        pred_xgb = pred.copy()

사용한 새순 Features 개수별 validation score
Feature 수 : 20 / 결과 : 29.986007674873605
Feature 수 : 25 / 결과 : 29.949397339019292
Feature 수 : 30 / 결과 : 29.945617941098888
Feature 수 : 35 / 결과 : 29.97863223858424
Feature 수 : 40 / 결과 : 29.987451800513604
Feature 수 : 45 / 결과 : 29.959170861324143
Feature 수 : 50 / 결과 : 29.92061504887735
Feature 수 : 55 / 결과 : 29.90271630090559
Feature 수 : 60 / 결과 : 29.916535260399705
Feature 수 : 65 / 결과 : 29.92043141882291
Feature 수 : 70 / 결과 : 29.94567816733882


In [ ]:
nmae_score(labels, oof_xgb)

0.0736116739440697

## 3-2. LightGBM

### 변수 중요도

In [ ]:
feature_columns = df_train.loc[:, "2022-09-01 새순":"2022-11-28 새순"].columns

feature_importance_df = pd.DataFrame()
oof = np.zeros(len(df_train_v9))
pred = np.zeros(len(df_test_v9))

for trn_idx, val_idx in kfold.split(df_train_v9):
    X_train = df_train_v9.loc[trn_idx, feature_columns].reset_index(drop=True)
    y_train = df_train_v9.loc[trn_idx, "착과량(int)"].reset_index(drop=True)
    X_valid = df_train_v9.loc[val_idx, feature_columns].reset_index(drop=True)
    y_valid = df_train_v9.loc[val_idx, "착과량(int)"].reset_index(drop=True)

    lr = LGBMRegressor(
        n_estimators=50000,
        learning_rate=0.01,
        subsample=0.7,
        colsample_bytree=0.3,
        random_state=42,
        )
    lr.fit(X_train, y_train, 
    eval_set=[(X_train, y_train), (X_valid, y_valid)], 
    early_stopping_rounds=500, 
    verbose=1000)
    
    oof[val_idx] = lr.predict(X_valid)
    pred += lr.predict(df_test[feature_columns]) / FOLDS
    feature_importance_df = pd.concat([feature_importance_df, 
                                       pd.DataFrame(zip(feature_columns, lr.feature_importances_), columns=["feature", "importance"])])
else:
    feature_importance_df = feature_importance_df.groupby("feature").mean().sort_values("importance", ascending=False).reset_index()
    print(nmae_score(labels, oof), mean_absolute_error(labels, oof))

Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[426]	training's l2: 951.335	valid_1's l2: 1584.66
Training until validation scores don't improve for 500 rounds.
[1000]	training's l2: 769.71	valid_1's l2: 1428.47
Early stopping, best iteration is:
[558]	training's l2: 922.865	valid_1's l2: 1394.64
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[362]	training's l2: 1063.21	valid_1's l2: 1334.17
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[431]	training's l2: 976.194	valid_1's l2: 1395.44
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[483]	training's l2: 965.637	valid_1's l2: 1329.56
0.07400102381726167 30.0608789696244


In [ ]:
best_score = 999999
best_cnt = 999999

for cnt in [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]:
    feature_columns = feature_importance_df.iloc[:cnt, 0].tolist()
    oof = np.zeros(len(df_train_v9))
    pred = np.zeros(len(df_test_v9))

    for trn_idx, val_idx in kfold.split(df_train_v9):
        X_train = df_train_v9.loc[trn_idx, feature_columns].reset_index(drop=True)
        y_train = df_train_v9.loc[trn_idx, "착과량(int)"].reset_index(drop=True)
        X_valid = df_train_v9.loc[val_idx, feature_columns].reset_index(drop=True)
        y_valid = df_train_v9.loc[val_idx, "착과량(int)"].reset_index(drop=True)

        lr = LGBMRegressor(
            n_estimators=50000,
            learning_rate=0.01,
            subsample=0.7,
            colsample_bytree=0.3,
            random_state=42,
            )
        lr.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_valid, y_valid)], 
        early_stopping_rounds=500, 
        verbose=1000)
        
        oof[val_idx] = lr.predict(X_valid)
        pred += lr.predict(df_test_v9[feature_columns]) / FOLDS
    
    print(cnt, mean_absolute_error(labels, oof))
    if best_score>mean_absolute_error(labels, oof):
        best_score = mean_absolute_error(labels, oof)
        oof_lgb = oof.copy()
        pred_lgb = pred.copy()

Training until validation scores don't improve for 500 rounds.
[1000]	training's l2: 981.048	valid_1's l2: 1554.04
Early stopping, best iteration is:
[504]	training's l2: 1065.24	valid_1's l2: 1538.1
Training until validation scores don't improve for 500 rounds.
[1000]	training's l2: 996.915	valid_1's l2: 1405.54
Early stopping, best iteration is:
[551]	training's l2: 1081.3	valid_1's l2: 1374.73
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[385]	training's l2: 1160.54	valid_1's l2: 1286.49
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[442]	training's l2: 1108.49	valid_1's l2: 1360.19
Training until validation scores don't improve for 500 rounds.
[1000]	training's l2: 1018.45	valid_1's l2: 1347.8
Early stopping, best iteration is:
[543]	training's l2: 1094.96	valid_1's l2: 1332.89
20 29.85874829489679
Training until validation scores don't improve for 500 rounds.
[1000]	training's 

In [ ]:
nmae_score(labels, oof_lgb)

0.07342039282325025

In [ ]:
nmae_score(labels, (oof_xgb + oof_lgb)/2)

0.07329980098327707

## 3-3. TabNet

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.augmentations import RegressionSMOTE

oof_nn = np.zeros(len(df_train_v9))
pred_nn = np.zeros(len(df_test_v9))


feature_columns= ["2022-09-02 새순", "2022-09-07 새순", "2022-09-05 새순", "2022-09-04 새순",
       "2022-09-11 새순", "2022-09-08 새순", "2022-09-16 새순", "2022-09-03 새순",
       "2022-09-10 새순", "2022-09-15 새순", "2022-09-14 새순", "2022-09-01 새순",
       "2022-09-06 새순", "2022-09-09 새순", "2022-09-13 새순", "2022-09-20 새순",
       "2022-09-18 새순", "2022-09-12 새순", "2022-09-17 새순", "2022-09-26 새순",
       "2022-09-19 새순", "2022-09-22 새순", "2022-10-03 새순", "2022-09-24 새순",
       "2022-09-30 새순"] # feature_importance top 31

for trn_idx, val_idx in kfold.split(df_train_v9):
    X_train = df_train_v9.loc[trn_idx, feature_columns].values
    y_train = df_train_v9.loc[trn_idx, "착과량(int)"].values.reshape(-1, 1)
    X_valid = df_train_v9.loc[val_idx, feature_columns].values
    y_valid = df_train_v9.loc[val_idx, "착과량(int)"].values.reshape(-1, 1)

    reg = TabNetRegressor(
        n_d=64, # [8, 64]
        n_a=64, # [8, 64]
        n_steps=10, # [3, 10]
        seed=42,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=1e-2),
        scheduler_params={"step_size":10, # how to use learning rate scheduler
                        "gamma":0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type="sparsemax" # This will be overwritten if using pretrain model
    )

    reg.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=["train", "valid"],
        loss_fn=torch.nn.L1Loss(),
        eval_metric=["mae"],
        max_epochs=300,
        batch_size=64,
        patience=50,
    )

    oof_nn[val_idx] = reg.predict(X_valid).reshape(-1)
    pred_nn += reg.predict(df_test_v9[feature_columns].values).reshape(-1) / FOLDS

epoch 0  | loss: 371.13492| train_mae: 293.85232| valid_mae: 306.27119|  0:00:03s
epoch 1  | loss: 289.2523| train_mae: 315.56309| valid_mae: 337.05035|  0:00:06s
epoch 2  | loss: 117.96897| train_mae: 253.52878| valid_mae: 266.90519|  0:00:09s
epoch 3  | loss: 61.59561| train_mae: 213.28244| valid_mae: 220.78068|  0:00:12s
epoch 4  | loss: 56.9967 | train_mae: 218.9324| valid_mae: 224.04901|  0:00:16s
epoch 5  | loss: 51.16617| train_mae: 190.77056| valid_mae: 196.52141|  0:00:19s
epoch 6  | loss: 54.21815| train_mae: 144.54276| valid_mae: 146.9346|  0:00:22s
epoch 7  | loss: 53.10071| train_mae: 91.17947| valid_mae: 90.28697|  0:00:25s
epoch 8  | loss: 53.41776| train_mae: 84.6763 | valid_mae: 86.86238|  0:00:29s
epoch 9  | loss: 49.09168| train_mae: 67.94374| valid_mae: 70.72718|  0:00:32s
epoch 10 | loss: 48.05256| train_mae: 57.77071| valid_mae: 58.42053|  0:00:35s
epoch 11 | loss: 47.28917| train_mae: 46.14104| valid_mae: 47.98617|  0:00:37s
epoch 12 | loss: 46.81217| train_mae: 

In [ ]:
nmae_score(labels, oof_nn)

0.07827946704068348

In [ ]:
nmae_score(labels, (oof_xgb*0.7 + oof_lgb*0.3)*0.95 + oof_nn*0.05)

0.07337539867730943